<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB42_Single_vs_Multi_Agent_version_compl%C3%A8te.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB42 : Mono Agent versus Multi Agent Interactions

**Objectif:** Expérimenter comment plusieurs agents légers coordonnent (ou entrent en conflit) par rapport à un agent unique accomplissant la même tâche.

**Durée estimée:** 15–20 minutes

**Livrable:** Un notebook qui exécute la même tâche en modes (A) single-agent et (B) multi-agent et compare les résultats.

---

## Architecture

- **Blackboard** (mémoire partagée)
- **Rôles** spécialisés (Researcher, Planner, Critic)
- **Protocole** de messages structuré
- **Comparaison** single-agent vs multi-agent

---

## Améliorations de cette version

✅ **Support multi-rounds fonctionnel**
✅ **Contexte enrichi**
✅ **Logs détaillés**
✅ **Comparaison complète des 3 approches**

## 1) Setup

In [1]:
!pip install -q openai python-dotenv

### Configuration API

In [2]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## 2) Agent Interface

In [3]:
import os, uuid, time
from dataclasses import dataclass, field
from typing import Dict, Any, List
from openai import OpenAI

llm = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def chat(system: str, user: str, temp: float = 0) -> str:
    r = llm.chat.completions.create(
        model="gpt-4o-mini",
        temperature=temp,
        messages=[{"role":"system","content":system},{"role":"user","content":user}]
    )
    return r.choices[0].message.content.strip()

@dataclass
class Message:
    sender: str
    content: str
    ts: float = field(default_factory=time.time)

@dataclass
class Blackboard:
    data: Dict[str, Any] = field(default_factory=dict)
    log: List[Message] = field(default_factory=list)
    def write(self, msg: Message): self.log.append(msg)
    def update(self, **kwargs): self.data.update(kwargs)

print("✓ Structures définies")

✓ Structures définies


## 3) Prompts pour chaque Rôle :

In [4]:
ROLE_RESEARCHER = (
    "Role: Researcher. Read the task & propose 3 concise facts or sources "
    "to inform a solution. Output as bullet points. No final answer."
)

ROLE_PLANNER = (
    "Role: Planner. Read the latest facts and propose an ordered plan with 3–5 steps. "
    "State assumptions and risks briefly."
)

ROLE_CRITIC = (
    "Role: Critic. Review the plan for gaps, conflicts, or missing data. "
    "Return 2–3 actionable improvements. No final answer."
)

ROLE_SINGLE = (
    "You are a single agent acting as Researcher, Planner, and Critic at once. "
    "Given the task, produce facts, a plan, then self-critique with improvements."
)

print("✓ Rôles définis")

✓ Rôles définis


## 4) Définition de la classe pour les agents :

In [5]:
class BaseAgent:
    def __init__(self, name: str, role: str):
        self.name, self.role = name, role

    def act(self, task: str, bb: Blackboard) -> Message:
        context = (
            f"Task:\n{task}\n\nBlackboard:\n{bb.data}\n\n"
            f"Conversation so far:\n" +
            "\n".join([f"- {m.sender}: {m.content[:200]}" for m in bb.log[-6:]])
        )
        out = chat(system=self.role, user=context)
        return Message(sender=self.name, content=out)

researcher = BaseAgent("Researcher", ROLE_RESEARCHER)
planner = BaseAgent("Planner", ROLE_PLANNER)
critic = BaseAgent("Critic", ROLE_CRITIC)
single = BaseAgent("Solo", ROLE_SINGLE)

print("✓ Agents instanciés")

✓ Agents instanciés


## 5) Boucle multi-Agents :

In [6]:
def summarize_last(msgs: List[Message], role: str) -> str:
    if not msgs: return ""
    joined = "\n".join([m.content for m in msgs[-4:]])
    return chat(system=f"Summarize key points for {role}.", user=joined)

def multi_agent_run(task: str, rounds: int = 1, verbose: bool = True) -> Blackboard:
    bb = Blackboard(data={"task": task, "facts": [], "plans": [], "critiques": [], "plan": "", "critique": ""})

    for r in range(rounds):
        if verbose:
            print(f"\n{'='*70}\n🔄 ROUND {r+1}/{rounds}\n{'='*70}")

        # 1) Researcher
        if verbose: print("  📚 Researcher: collecte des faits...")
        researcher_context = task
        if r > 0:
            researcher_context += (
                f"\n\nPrevious facts collected:\n" +
                "\n".join([f"- {f[:100]}..." for f in bb.data["facts"][-3:]]) +
                "\n\nProvide NEW complementary facts or deeper insights."
            )
        m1 = researcher.act(researcher_context, bb)
        bb.write(m1)
        bb.data["facts"].append(m1.content)
        if verbose: print(f"    ✓ {len(m1.content)} caractères")

        # 2) Planner
        if verbose: print("  📋 Planner: création du plan...")
        ctx = summarize_last(bb.log, "Planner")
        planner_prompt = task + "\n\nContext:\n" + ctx
        if r > 0:
            planner_prompt += (
                f"\n\nPrevious plan (Round {r}):\n{bb.data['plans'][-1][:200]}..."
                "\n\nRefine and improve based on new facts."
            )
        m2 = planner.act(planner_prompt, bb)
        bb.write(m2)
        bb.data["plan"] = m2.content
        bb.data["plans"].append(m2.content)
        if verbose: print(f"    ✓ Plan v{r+1} ({len(m2.content)} caractères)")

        # 3) Critic
        if verbose: print("  🔍 Critic: analyse et critique...")
        ctx = summarize_last(bb.log, "Critic")
        m3 = critic.act(task + "\n\nContext:\n" + ctx, bb)
        bb.write(m3)
        bb.data["critique"] = m3.content
        bb.data["critiques"].append(m3.content)
        if verbose: print(f"    ✓ Critique ({len(m3.content)} caractères)")

        # 4) Planner révision
        if verbose: print("  ✏️  Planner: révision...")
        ctx = summarize_last(bb.log, "Planner")
        m4 = planner.act(task + "\n\nRevise plan per critique:\n" + ctx, bb)
        bb.write(m4)
        bb.data["plan"] = m4.content
        if verbose: print(f"    ✓ Plan révisé ({len(m4.content)} caractères)")

        bb.update(round=r+1)
        if verbose:
            print(f"\n  📊 Round {r+1} terminé:")
            print(f"    - Messages: {len(bb.log)}")
            print(f"    - Faits: {len(bb.data['facts'])}")
            print(f"    - Plans: {len(bb.data['plans'])}")

    if verbose:
        print(f"\n{'='*70}\n✅ Workflow terminé\n{'='*70}")
        print(f"📈 Statistiques: {rounds} round(s), {len(bb.log)} messages, {len(bb.data['facts'])} faits")
    return bb

print("✓ Fonction multi_agent_run AMÉLIORÉE")

✓ Fonction multi_agent_run AMÉLIORÉE


## 6) Mode mono/single-Agent :

In [7]:
def single_agent_run(task: str) -> Blackboard:
    bb = Blackboard(data={"task": task})
    print("\n🤖 Single-agent: traitement...")
    m = single.act(task, bb)
    bb.write(m)
    bb.update(final=m.content)
    print("✓ Single-agent terminé")
    return bb

print("✓ Fonction single_agent_run définie")

✓ Fonction single_agent_run définie


## 7) Exécution des 3 approches

In [9]:
TASK = (
    "Design a 3-email outreach sequence for a B2B AI tool targeting "
    "healthcare analytics leaders. Include subject lines and a clear CTA each. Give the result in French."
)

print("="*70)
print("TÂCHE À ACCOMPLIR")
print("="*70)
print(TASK)
print("="*70)

TÂCHE À ACCOMPLIR
Design a 3-email outreach sequence for a B2B AI tool targeting healthcare analytics leaders. Include subject lines and a clear CTA each. Give the result in French.


### Test 1: Multi-agent avec 1 seul tour (round) :

In [10]:
bb_multi_1 = multi_agent_run(TASK, rounds=1)


🔄 ROUND 1/1
  📚 Researcher: collecte des faits...
    ✓ 819 caractères
  📋 Planner: création du plan...
    ✓ Plan v1 (2290 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique (1135 caractères)
  ✏️  Planner: révision...
    ✓ Plan révisé (2735 caractères)

  📊 Round 1 terminé:
    - Messages: 4
    - Faits: 1
    - Plans: 1

✅ Workflow terminé
📈 Statistiques: 1 round(s), 4 messages, 1 faits


### Test 2: Multi-agent avec 2 tours :

In [11]:
bb_multi_2 = multi_agent_run(TASK, rounds=2)


🔄 ROUND 1/2
  📚 Researcher: collecte des faits...
    ✓ 732 caractères
  📋 Planner: création du plan...
    ✓ Plan v1 (2159 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique (1211 caractères)
  ✏️  Planner: révision...
    ✓ Plan révisé (2650 caractères)

  📊 Round 1 terminé:
    - Messages: 4
    - Faits: 1
    - Plans: 1

🔄 ROUND 2/2
  📚 Researcher: collecte des faits...
    ✓ 871 caractères
  📋 Planner: création du plan...
    ✓ Plan v2 (2853 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique (1067 caractères)
  ✏️  Planner: révision...
    ✓ Plan révisé (2953 caractères)

  📊 Round 2 terminé:
    - Messages: 8
    - Faits: 2
    - Plans: 2

✅ Workflow terminé
📈 Statistiques: 2 round(s), 8 messages, 2 faits


### Test 3: Mode Single-agent : plusieurs étapes réalisées par le même agent :

In [12]:
bb_single = single_agent_run(TASK)


🤖 Single-agent: traitement...
✓ Single-agent terminé


## 8) Affichage des résultats

### Résultats Multi-Agent 1 ROUND

In [13]:
print("\n" + "="*70)
print("RÉSULTATS MULTI-AGENT - 1 ROUND")
print("="*70)

print("\n📊 Statistiques:")
print(f"  - Messages: {len(bb_multi_1.log)}")
print(f"  - Faits: {len(bb_multi_1.data['facts'])}")
print(f"  - Plans: {len(bb_multi_1.data['plans'])}")

print("\n📚 FAITS:")
print("-" * 70)
for i, fact in enumerate(bb_multi_1.data.get("facts", []), 1):
    print(f"\nFait {i}:\n{fact}")

print("\n" + "="*70)
print("📋 PLAN FINAL:")
print("="*70)
print(bb_multi_1.data.get("plan", ""))

print("\n" + "="*70)
print("🔍 CRITIQUE:")
print("="*70)
print(bb_multi_1.data.get("critique", ""))


RÉSULTATS MULTI-AGENT - 1 ROUND

📊 Statistiques:
  - Messages: 4
  - Faits: 1
  - Plans: 1

📚 FAITS:
----------------------------------------------------------------------

Fait 1:
- **Fact 1**: The healthcare analytics market is projected to grow significantly, with a focus on improving patient outcomes and operational efficiency, making it a prime target for AI tools. (Source: MarketsandMarkets report on healthcare analytics)

- **Fact 2**: Personalization in email outreach can increase engagement rates, especially when addressing specific pain points faced by healthcare analytics leaders, such as data integration and predictive analytics. (Source: HubSpot's Email Marketing Statistics)

- **Fact 3**: Clear and compelling calls-to-action (CTAs) in emails can significantly improve response rates, with actionable language that encourages the recipient to take the next step, such as scheduling a demo or downloading a whitepaper. (Source: Campaign Monitor's Email Marketing Best Practices

### Résultats Multi-Agent 2 ROUNDS

In [14]:
print("\n" + "="*70)
print("RÉSULTATS MULTI-AGENT - 2 ROUNDS")
print("="*70)

print("\n📊 Statistiques:")
print(f"  - Messages: {len(bb_multi_2.log)}")
print(f"  - Faits: {len(bb_multi_2.data['facts'])}")
print(f"  - Plans: {len(bb_multi_2.data['plans'])}")

print("\n📚 ÉVOLUTION DES FAITS:")
print("-" * 70)
for i, fact in enumerate(bb_multi_2.data.get("facts", []), 1):
    print(f"\n[Round {i}] Fait {i}:\n{fact}")

print("\n" + "="*70)
print("📋 PLAN FINAL (après 2 rounds):")
print("="*70)
print(bb_multi_2.data.get("plan", ""))

print("\n" + "="*70)
print("🔍 CRITIQUE FINALE:")
print("="*70)
print(bb_multi_2.data.get("critique", ""))


RÉSULTATS MULTI-AGENT - 2 ROUNDS

📊 Statistiques:
  - Messages: 8
  - Faits: 2
  - Plans: 2

📚 ÉVOLUTION DES FAITS:
----------------------------------------------------------------------

[Round 1] Fait 1:
- **Fact 1**: The healthcare analytics market is projected to grow significantly, with a focus on improving patient outcomes and operational efficiency, making it a prime target for AI tools. (Source: MarketsandMarkets report on healthcare analytics)

- **Fact 2**: Personalization in email outreach can increase engagement rates; tailoring messages to specific pain points of healthcare analytics leaders can enhance response rates. (Source: HubSpot's Email Marketing Statistics)

- **Fact 3**: Clear and compelling calls-to-action (CTAs) are crucial in B2B email campaigns, as they guide recipients towards the desired action, such as scheduling a demo or downloading a whitepaper. (Source: Mailchimp's Email Marketing Benchmarks)

[Round 2] Fait 2:
- **Fact 4**: The integration of AI in he

### Résultats SINGLE-AGENT

In [15]:
print("\n" + "="*70)
print("RÉSULTATS SINGLE-AGENT")
print("="*70)

print("\n📊 Statistiques:")
print(f"  - Messages: {len(bb_single.log)}")

print("\n" + "="*70)
print("🤖 OUTPUT COMPLET:")
print("="*70)
print(bb_single.data.get("final", ""))


RÉSULTATS SINGLE-AGENT

📊 Statistiques:
  - Messages: 1

🤖 OUTPUT COMPLET:
### Email Outreach Sequence for B2B AI Tool Targeting Healthcare Analytics Leaders

#### Email 1: Introduction

**Subject Line:** Transformez vos analyses de santé avec notre outil IA

**Body:**
Bonjour [Nom],

Je suis [Votre Nom] de [Votre Entreprise], et je suis ravi de vous présenter notre outil d'analyse basé sur l'IA, conçu spécifiquement pour les leaders en santé. Notre solution permet d'optimiser les processus d'analyse de données, d'améliorer la prise de décision et d'accroître l'efficacité opérationnelle.

Nous avons aidé des organisations comme la vôtre à transformer leurs données en informations exploitables, ce qui a conduit à des résultats significatifs en matière de soins aux patients.

**CTA:** Êtes-vous disponible pour une démonstration de 20 minutes cette semaine ? Répondez à cet e-mail pour convenir d'un créneau.

---

#### Email 2: Suivi et Témoignage

**Subject Line:** Découvrez comment [Nom

## 9) Comparaison finale des 3 approches

In [16]:
print("\n" + "="*70)
print("COMPARAISON STATISTIQUES")
print("="*70)

print("\n📊 Multi-agent 1 round:")
print(f"  - Messages: {len(bb_multi_1.log)}")
print(f"  - Faits: {len(bb_multi_1.data['facts'])}")
print(f"  - Plans: {len(bb_multi_1.data['plans'])}")
print(f"  - Longueur: {len(bb_multi_1.data.get('plan', ''))} caractères")

print("\n📊 Multi-agent 2 rounds:")
print(f"  - Messages: {len(bb_multi_2.log)}")
print(f"  - Faits: {len(bb_multi_2.data['facts'])}")
print(f"  - Plans: {len(bb_multi_2.data['plans'])}")
print(f"  - Longueur: {len(bb_multi_2.data.get('plan', ''))} caractères")

print("\n📊 Single-agent:")
print(f"  - Messages: {len(bb_single.log)}")
print(f"  - Longueur: {len(bb_single.data.get('final', ''))} caractères")

print("\n" + "="*70)
print("RATIO D'EFFORT")
print("="*70)
print(f"Multi-agent 1 round: {len(bb_multi_1.log)}x appels vs single-agent")
print(f"Multi-agent 2 rounds: {len(bb_multi_2.log)}x appels vs single-agent")
print(f"Amélioration 2 rounds vs 1: {len(bb_multi_2.log)/len(bb_multi_1.log):.1f}x appels")


COMPARAISON STATISTIQUES

📊 Multi-agent 1 round:
  - Messages: 4
  - Faits: 1
  - Plans: 1
  - Longueur: 2735 caractères

📊 Multi-agent 2 rounds:
  - Messages: 8
  - Faits: 2
  - Plans: 2
  - Longueur: 2953 caractères

📊 Single-agent:
  - Messages: 1
  - Longueur: 3383 caractères

RATIO D'EFFORT
Multi-agent 1 round: 4x appels vs single-agent
Multi-agent 2 rounds: 8x appels vs single-agent
Amélioration 2 rounds vs 1: 2.0x appels


### Évaluation qualitative automatique

In [17]:
comparison_prompt = f"""
Compare three approaches for: {TASK}

[Approach A: Multi-agent 1 round]
{bb_multi_1.data.get('plan', '')}

[Approach B: Multi-agent 2 rounds]
{bb_multi_2.data.get('plan', '')}

[Approach C: Single-agent]
{bb_single.data.get('final', '')}

Score each (1-5) on: Coverage, Quality, Depth, Structure
Then: Which is best and why? (3 sentences) When to use each? (2 sentences each)
Reply all in french
"""

print("\n" + "="*70)
print("ÉVALUATION QUALITATIVE")
print("="*70)
evaluation = chat("You are a fair evaluator.", comparison_prompt)
print(evaluation)


ÉVALUATION QUALITATIVE
### Évaluation des Approches

#### Approche A
- **Couverture**: 4
- **Qualité**: 4
- **Profondeur**: 3
- **Structure**: 4

#### Approche B
- **Couverture**: 5
- **Qualité**: 5
- **Profondeur**: 4
- **Structure**: 5

#### Approche C
- **Couverture**: 3
- **Qualité**: 3
- **Profondeur**: 3
- **Structure**: 3

### Meilleure Approche
L'Approche B est la meilleure car elle offre une couverture complète des points de douleur, une qualité de contenu élevée et une structure bien définie qui guide le lecteur à travers le processus de décision. Elle utilise également un suivi efficace pour maintenir l'engagement, ce qui est crucial dans un contexte B2B. 

### Quand utiliser chaque approche
- **Approche A**: Utilisez cette approche lorsque vous souhaitez une séquence d'emails concise et directe, idéale pour des cibles qui ont déjà une certaine familiarité avec les outils d'IA et qui apprécient une communication rapide.
- **Approche B**: Cette approche est recommandée lorsq

### Analyse amélioration incrémentale

In [18]:
print("\n" + "="*70)
print("ANALYSE: AMÉLIORATION ROUND 1 → ROUND 2")
print("="*70)

if len(bb_multi_2.data['plans']) >= 2:
    print("\n📋 PLAN Round 1 (extrait):")
    print("-" * 70)
    print(bb_multi_2.data['plans'][0][:400] + "...")

    print("\n📋 PLAN Round 2 (extrait):")
    print("-" * 70)
    print(bb_multi_2.data['plan'][:400] + "...")

    diff_prompt = f"""
Compare these two versions:

[Version 1 - Round 1]
{bb_multi_2.data['plans'][0]}

[Version 2 - Round 2]
{bb_multi_2.data['plan']}

List 3-5 specific improvements in Version 2. Be concrete.
"""

    print("\n" + "="*70)
    print("🔍 AMÉLIORATIONS IDENTIFIÉES:")
    print("="*70)
    improvements = chat("You are an analyst.", diff_prompt)
    print(improvements)


ANALYSE: AMÉLIORATION ROUND 1 → ROUND 2

📋 PLAN Round 1 (extrait):
----------------------------------------------------------------------
### Plan for Email Outreach Sequence

**Assumptions:**
- The target audience consists of healthcare analytics leaders who are looking for innovative solutions to improve patient outcomes and operational efficiency.
- The AI tool being promoted has features that directly address common pain points in healthcare analytics.

**Risks:**
- Low engagement rates if emails are not personalized effectively...

📋 PLAN Round 2 (extrait):
----------------------------------------------------------------------
### Plan for Email Outreach Sequence

**Assumptions:**
- The target audience consists of healthcare analytics leaders who are looking for innovative solutions to improve patient outcomes and operational efficiency.
- The AI tool being promoted has features that directly address common pain points in healthcare analytics.

**Risks:**
- Low engagement rates i

## 10) Conclusion

### 🎯 Enseignements clés

**Différences structurelles:**
- Single-agent: 1 appel LLM
- Multi-agent 1 round: 4 appels (Research→Plan→Critique→Révision)
- Multi-agent 2 rounds: 8 appels (2 cycles)

**Trade-offs:**
- **Coût**: Multi-agent = N × appels
- **Latence**: Appels séquentiels
- **Qualité**: Généralement meilleure pour tâches complexes
- **Robustesse**: La critique rattrape les erreurs

### 📚 Principes multi-agents

1. **Stateless LLMs**: Sans historique explicite, chaque appel isolé
2. **Context is king**: Plus de contexte = meilleure continuité
3. **Spécialisation**: Rôles > généralistes
4. **Feedback loops**: Critique = amélioration itérative
5. **Blackboard**: Mémoire partagée pour coordination

### 🚀 Quand utiliser?

**Single-agent**: Tâches simples, budget limité, prototypage

**Multi-agent 1 round**: Expertise multiple, validation requise

**Multi-agent N rounds**: Complexe, qualité max, raffinement

⚠️ **Attention**: Rendements décroissants après 2-3 rounds

## 11) Expérimentations optionnelles

### Test avec 3 rounds

In [ ]:
# Décommentez pour tester
# bb_multi_3 = multi_agent_run(TASK, rounds=3)
# print(f"Stats 3 rounds: {len(bb_multi_3.log)} messages")
# print(f"\nPlan:\n{bb_multi_3.data['plan']}")

### Tâche alternative

In [ ]:
TASK_2 = "Create a 5-step incident response checklist for an LLM API outage."

# Décommentez
# bb_alt_multi = multi_agent_run(TASK_2, rounds=2)
# bb_alt_single = single_agent_run(TASK_2)

### Estimation de coût

In [ ]:
def estimate_tokens(text: str) -> int:
    return len(text) // 4

print("\n" + "="*70)
print("ESTIMATION DE COÛT")
print("="*70)

multi1_chars = sum(len(m.content) for m in bb_multi_1.log)
multi2_chars = sum(len(m.content) for m in bb_multi_2.log)
single_chars = sum(len(m.content) for m in bb_single.log)

print(f"\nOutput généré:")
print(f"  - 1 round: ~{estimate_tokens(multi1_chars):,} tokens")
print(f"  - 2 rounds: ~{estimate_tokens(multi2_chars):,} tokens")
print(f"  - Single: ~{estimate_tokens(single_chars):,} tokens")

output_price = 0.0006  # per 1K tokens
print(f"\n💰 Coût OUTPUT (~${output_price}/1K tokens):")
print(f"  - 1 round: ${estimate_tokens(multi1_chars)/1000*output_price:.4f}")
print(f"  - 2 rounds: ${estimate_tokens(multi2_chars)/1000*output_price:.4f}")
print(f"  - Single: ${estimate_tokens(single_chars)/1000*output_price:.4f}")